# <center>Gensim </center>
<center>Shan-Hao Yu, Tammy Chang &amp; IBMGBS Data Scientist Team<br/>Dec 2019</center>

## 文字檔(txt)的讀取與轉換

In [1]:
import os
import pandas as pd
import numpy as np
import string

In [2]:
TRAINING_PATH = 'dataset/'

In [3]:
with open(TRAINING_PATH+'data_1_cut.txt') as f:
    docs = f.readlines()

In [4]:
print ("Length of list: {}".format(len(docs)))
print ("Length of first docs: {}".format(len(docs[0])))

Length of list: 4529
Length of first docs: 24


In [5]:
docs[0]

' 1 . 患者 老年 女性 ， 88 歲 ； \n'

In [6]:
docs_list = []
for d in docs:
    token_list = d.strip().split(" ")
    del_list = []
    for t in token_list:
        if len(t) < 2:
            del_list.append(t)
        elif t.isdigit():
            del_list.append(t)
        elif t in string.punctuation:
            del_list.append(t)
    for d in del_list:
        token_list.remove(d)
    docs_list.append(token_list)

In [7]:
print ("Length of list: {}".format(len(docs_list)))
print ("Length of first docs: {}".format(len(docs_list[0])))

Length of list: 4529
Length of first docs: 3


In [8]:
docs_list[0]

['患者', '老年', '女性']

# 〈Gensim Word2Vec 簡易教學〉

### Word2Vec 的 input 可以是 1D list 或 2D list（亦可直接使用 numpy 或 pandas 的結構）
> **1D list：**[ '我愛林書豪', '你是周杰倫' ] → 會被 Word2Vec 以單個字元分詞成 [[ '我', '愛', '林', '書' , '豪' ], [ '你', '是', '周', '杰' , '倫' ]] 來訓練 <br>
> **2D list：**[[ '我', '愛', '林書豪' ], [ '你', '是' , '周杰倫' ]] →  Word2Vec 會直接使用我們分好的詞來訓練

### Q1: 我們應該將分詞好的文本處理成哪種資料結構呢？


### Step1: 要先引入引用Gensim 中的 Word2Vec 模組

In [9]:
####################################################
############### Your Code Here #####################


from gensim.models.word2vec import Word2Vec


############### End of Code ########################
####################################################

### Step2: 把2D資料結構，直接扔進Word2Vec就大功告成啦！

In [10]:
####################################################
############### Your Code Here #####################


model = Word2Vec(docs_list)


############### End of Code ########################
####################################################

### 接下來我們要檢驗訓練好的詞向量模型
（先定義個函式，可以一次比較很多個詞的相似詞）

In [11]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

### Step3: 隨意挑選詞彙，檢視與其最相似的10個詞

In [12]:
most_similar(model, ['手術', '頭痛', '高血壓', '患者', '飲食'])

,手術,cos,頭痛,cos,高血壓,cos,患者,cos,飲食,cos
0,切除,0.883152,頭暈,0.989464,患高血壓,0.970171,根據,0.902671,睡眠,0.964288
1,曾行,0.873097,發作性,0.941589,余年,0.962972,出院,0.897232,二便,0.940607
2,結核病,0.862889,痛頭暈,0.941290,患有,0.959479,入院,0.888514,尿量,0.934845
3,健骨,0.856224,當時,0.939297,糖尿病,0.946746,因車,0.874961,小便,0.879966
4,輸血史,0.842276,惡心,0.937660,病史,0.943863,青年,0.846627,精神,0.878993
5,疤痕,0.836724,無頭,0.932084,否認,0.941270,中年,0.838752,大便,0.857742
6,行闌尾,0.833616,憋氣,0.918995,肝炎,0.939690,女性,0.838130,尚可,0.815596
7,在院,0.832596,傷後頭,0.914943,無高血壓,0.933922,中老年,0.836613,狀況,0.803358
8,過敏,0.830678,暈頭,0.914864,既往,0.930451,本次,0.836194,以來,0.802982
9,應用,0.829067,旋轉,0.901741,服藥,0.929631,手續,0.834691,意識,0.802260


In [14]:
model.predict_output_word(['手術'],10)

[('血管', 0.0036724755),
 ('藥物', 0.0025589878),
 ('手術', 0.0018424459),
 ('過敏史', 0.0015976289),
 ('否認', 0.0015774762),
 ('血糖', 0.0014271232),
 ('血脂', 0.0014131888),
 ('擴張', 0.0012661027),
 ('過敏', 0.0012221286),
 ('輸血史', 0.0012156666)]

### Step4: 最後用 save() 與 load() 來存取訓練好的 Word2Vec 模型

In [15]:
####################################################
############### Your Code Here #####################


model.save('word2vec.model')


############### End of Code ########################
####################################################

## Load the model 
model = Word2Vec.load('word2vec.model')

<a id="guide"></a>
# 〈Gensim Word2Vec 參數調整〉
### 準則：
1.  請先準備一個訓練快速的陽春模型，先把feature處理到你滿意為止再開始調整模型 <br>
    feature決定了準確度的上限，調整模型只是讓我們盡可能接近這個上限而已 <br>
    
2.   請不要抱持對每個參數都亂調一遍的心態，合理的猜想可以節省很多時間 <br>
   比起去亂調參數，不如花時間去了解這些參數背後的理論 <br>
   
3.   請勿用你的肉眼來斷定詞向量模型的好壞，交給你的模型來判斷 <br>
   看起來很爛的詞向量，深度學習或許可以從中找出肉眼看不見的規則

### **size**=100：詞向量的維度大小，維度太小會無法有效表達詞與詞的關係，維度太大會使關係太稀疏而難以找出規則 
    在一些資料分析的比賽(Kaggle)上常用的詞向量維度介於200到300之間
### **iter=5**：訓練的回數，訓練過少會使得詞關係過為鬆散，訓練過度又會使得詞關係過為極端
    當使用較大的詞向量維度時，可能會需要訓練更多次。
### **sg**=0：sg=0時以CBOW來訓練，sg=1時以Skip-gram來訓練
    某些詞彙可能只出現在特定的主題當中，所以文本中可能有許多低頻詞
    而在特性上，Skip-gram比CBOW通常對低頻詞有更好的訓練效果
###  **window=5**：CBOW下決定Word2Vec一次取多少詞來預測中間詞（Skip-gram的狀況是反過來的）
    需要多少詞才能預測中間詞 or 要讓多少詞的含意來影響中間詞的含意
    思考完就自己作點實驗吧～
### **min_count=5**：出現次數大於等於min_count的詞，才會納入Word2Vec的詞典中
### **max_vocab_size=None**：Word2Vec的詞典容納上限，出現次數最低的詞會優先被剔除
    降低詞典的最大詞數，可能讓模型更容易抓到規則（噪音減少了），但也可能更難抓到規則（認識的詞太少）
### **hs=0**：hs=0時採用Negative Sampling，hs=1時採用Hierarchical Softmax
### **negative=5**：Negative Sampling的取樣數量，5~20適合小數據，2~5適合大數據
    這會直接影響權重更新的方法，原理有點複雜我不敢亂解釋，請自己去翻論文_(:з」∠)_ 
### **workers=3**：訓練用的線程數量（可以加快訓練速度）

### 如果是你，會朝什麼方向調整模型的參數呢？

In [ ]:
####################################################
############### Your Code Here #####################


model_2 = Word2Vec(docs_list,size=200,iter=8, min_count=50)
most_similar(model_2, ['手術', '頭痛', '高血壓', '患者', '飲食'])


############### End of Code ########################
####################################################

In [ ]:
model_2.predict_output_word(['手術'],10)